In [54]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, log_loss

In [2]:
train_data = pd.read_parquet("../data/unadultered/train_data.parquet")
test_data = pd.read_parquet("../data/unadultered/test_data.parquet").reset_index()

In [81]:
def clean_test_data(data = train_data, debug=True, y_value =True,):
    """Cleans data given in test_data schema

    Args:
        data (test_data schema, optional): Just the typical input data, accepts test_data and train_data. Defaults to test_data.

    Returns:
        _type_: the same train_data database with the appropriate datatypes
    """
    cleaned_data = data
    for col_num in range(1, 367):
        col = "f" + str(col_num)
        print(col+":") if debug else None
        # These columns have strings hence we want to change or drop them soon
        if col_num in [42, 50] + list(range(52, 57)):
            print("pass") if debug else None
            pass
        elif col_num in list(range(226, 310)) + list(range(332, 336)) + [359, 360]:
            cleaned_data[col] = data[col].astype(bool)
            print("bool") if debug else None
        else:
            col_data = pd.to_numeric(
                data[col], errors="coerce"
            )  # converts to numeric, keeps NaN
            # Check if any non-NaN value has decimal part
            if (col_data.dropna() % 1 != 0).any():
                cleaned_data[col] = col_data.astype(float)
                print("bool") if debug else None
            else:
                cleaned_data[col] = col_data.astype("Int64")
                print("Int") if debug else None
                
    # Lastly
    cleaned_data["y"] = data["y"].astype("Int64") if y_value else None
    cleaned_data = cleaned_data.fillna(0)
    return cleaned_data
    

In [75]:
cleanedData = clean_test_data(data=train_data, debug=False)
# Dropping the passed columns, these are the columns that have strings in their values.
cleanedData = cleanedData.drop(["f42","f50","f52","f53","f54","f55","f56"],axis=1)

In [76]:
# Splitting the data into the Xdata and Ydata
Xdata = cleanedData.iloc[:,6:]
Ydata = cleanedData.iloc[:,5]
# Splitting into training and testing amongst the training data just for simplicity
X_train, X_test, y_train, y_test = train_test_split(Xdata, Ydata, test_size=0.2)

In [77]:
# Now running the model at the moment, Linear Regression
model = LogisticRegression(max_iter=1000)
model.fit(X_train, y_train)

c:\Users\rohan\miniconda3\envs\pytorchEnv\Lib\site-packages\sklearn\linear_model\_logistic.py:470: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=1000).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,None
,solver,'lbfgs'
,max_iter,1000
,multi_class,'deprecated'


In [78]:
y_pred_proba = model.predict_proba(X_test)[:, 1]
print("AUC:", roc_auc_score(y_test, y_pred_proba))
print("Log Loss:", log_loss(y_test, y_pred_proba))

AUC: 0.7260507966040406
Log Loss: 0.18072669272474803


In [87]:
cleaned_test_data = clean_test_data(data=test_data,y_value=False, debug=False)
# Dropping the passed columns, these are the columns that have strings in their values.
cleaned_test_data = cleaned_test_data.drop(["f42","f50","f52","f53","f54","f55","f56","y"],axis=1)

C:\Users\rohan\AppData\Local\Temp\ipykernel_33600\1193620459.py:35: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  cleaned_data = cleaned_data.fillna(0)


In [88]:
XTestingActualData = cleaned_test_data.iloc[:,6:]
XTestingActualData.head()

,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,...,f357,f358,f359,f360,f361,f362,f363,f364,f365,f366
0,1,0,0,0,0,0,0,0,0,13,...,0.0,-9999.0,True,False,28,0,0.00000,337,0,0.00000
1,1,0,0,0,0,0,0,0,0,13,...,0.0,0.0,True,False,87,0,0.00000,1010,2,0.00198
2,1,0,0,0,0,0,0,0,0,13,...,0.0,0.0,True,False,23,0,0.00000,1010,2,0.00198
3,1,0,0,0,0,0,0,0,0,13,...,0.0,-9999.0,True,False,277,1,0.00361,337,0,0.00000
4,1,0,0,0,0,0,0,0,0,13,...,0.0,-9999.0,True,False,359,0,0.00000,337,0,0.00000


In [89]:
y_pred_test_proba = model.predict_proba(XTestingActualData)

In [90]:
y_pred_test_proba

array([[0.95816284, 0.04183716],
       [0.94599123, 0.05400877],
       [0.80432572, 0.19567428],
       ...,
       [0.99662852, 0.00337148],
       [0.99867777, 0.00132223],
       [0.99341277, 0.00658723]], shape=(369301, 2))

In [91]:
click_probs = y_pred_test_proba[:,1]
df_click_probs = pd.DataFrame({"click_probabability": click_probs})
df_click_probs.to_csv('click_probabilities.csv', index=False)

In [73]:
train_data["f58"].unique()

<IntegerArray>
[ <NA>,   804,  2719,   818,  6441,  6440,  3240,   300,  4663,  2041,
 ...
  1309,  3963,  7230, 13185,  1144,  6599, 10471,  6749,  6857,  6453]
Length: 7861, dtype: Int64